## Shopping Cart Data Model Using Redis

### Requirements
- Each user has their own cart
- Carts can hold 0 or more items (SKUs)
- Each SKU can have a quantity > 1
- Each cart is linked to a user

---

### Redis Data Structure: Hash (for User Profiles)

We store user profiles as Redis hashes:
- **Key**: `user:<user_id>`
- **Fields**: name, email, and other user attributes

**Example:**
user:123 = { "name": "Saurabh", "email": "saurabh@example.com" }


**Why Hash?**
- Clean key–value structure
- Efficient to update and retrieve individual fields
- Supports optional fields without creating multiple keys
- Constant-time operations for single field access (`HGET`, `HSET`, etc.)

---

### Redis Data Structure: Hash (for Shopping Carts)

Each shopping cart is also stored as a Redis hash:
- **Key**: `cart:<user_id>`
- **Fields**: SKUs (product IDs)
- **Values**: quantities of each SKU

**Example:**

cart:123 = { "SKU123": "2", "SKU999": "1" }


**Why Hash?**
- Efficient storage of many field–value pairs under one key
- Quantities can be atomically updated using `HINCRBY`
- Easy to query (`HGET`, `HGETALL`, `HDEL`)
- Minimal memory usage and good performance for small to medium-sized carts

---




## Setup and and connect to Redis 

In [2]:
import redis

r = redis.Redis(
    host=os.environ['REDIS_SERVER1_IP'],
    port=6379,
    password=os.environ['REDIS_SERVER1_PASSWORD'],
    decode_responses=True
)

try:
    pong = r.ping()
    print("Connected to Redis:", pong)
except redis.AuthenticationError:
    print("Wrong password")
except Exception as e:
    print("Connection failed:", str(e))


Connected to Redis: True


In [3]:
def add_user(user_id, name, email):
    r.hset(f"user:{user_id}", mapping={"name": name, "email": email})

In [4]:
add_user("123", "Saurabh", "Saurabh@example.com")

In [5]:
print(r.hgetall("user:123"))

{'name': 'Saurabh', 'email': 'Saurabh@example.com'}


In [6]:
def add_to_cart(user_id, sku, quantity=1):
    r.hincrby(f"cart:{user_id}", sku, quantity)

add_to_cart("123", "SKU123", 2)
add_to_cart("123", "SKU999", 3)

In [7]:
print(r.hgetall("cart:123"))

{'SKU123': '2', 'SKU999': '3'}


In [8]:
def view_cart(user_id):
    return r.hgetall(f"cart:{user_id}")

view_cart("123")

{'SKU123': '2', 'SKU999': '3'}

In [9]:
print(r.hgetall("user:123"))

{'name': 'Saurabh', 'email': 'Saurabh@example.com'}


In [10]:
#remove items from the cart
def remove_from_cart(user_id, sku, quantity=1):
    current_qty = int(r.hget(f"cart:{user_id}", sku) or 0)
    new_qty = max(current_qty - quantity, 0)
    
    if new_qty == 0:
        r.hdel(f"cart:{user_id}", sku)
    else:
        r.hset(f"cart:{user_id}", sku, new_qty)

remove_from_cart("123", "SKU123", 1)


In [11]:
#clear cart
def clear_cart(user_id):
    r.delete(f"cart:{user_id}")

clear_cart("123")


In [12]:
print(r.hgetall("user:123"))

{'name': 'Saurabh', 'email': 'Saurabh@example.com'}


In [13]:
print(r.hgetall("cart:123"))

{}
